In [1]:
from corus import load_lenta
import re
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer,  TFBertModel, MBartTokenizer, MBartForConditionalGeneration
import annoy
import time

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru",)
model = TFBertModel.from_pretrained("sberbank-ai/sbert_large_mt_nlu_ru")
vector_size = 1024

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

C:\Users\nikit\.conda\envs\NLP\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nikit\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at sberbank-ai/sbert_large_mt_nlu_ru.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [3]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

In [4]:
re.sub('\xa0', ' ', next(records).text)

'Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.'

In [5]:
article_lenght = 128
article_index = annoy.AnnoyIndex(vector_size, 'angular')
index_map = {}
counter = 0

nw = time.time()
for record in load_lenta(path):

    index_map[counter] = re.sub('\xa0', ' ', record.text)

    tokens = tokenizer([index_map[counter]], max_length=article_lenght, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    vector = model(**tokens)


    token_embeddings = vector[0][0]
    input_mask = tokens['attention_mask'][0]
    sum_embeddings = token_embeddings[input_mask == 1]
    embed = tf.reduce_sum(sum_embeddings, 0)/tf.reduce_sum(tf.cast(input_mask, dtype=tf.float32))

    article_index.add_item(counter, embed)

    counter += 1

    if (time.time() - nw) // 60 > 30:
        break

article_index.build(10)

True

In [6]:
def get_response(model, tokenizer, question, article_index,  answers_nmb=5):

    tokens = tokenizer([question], max_length=article_lenght, truncation=True, padding='max_length', return_token_type_ids=False, return_tensors='tf')
    vector = model(**tokens)


    token_embeddings = vector[0][0]
    input_mask = tokens['attention_mask'][0]
    sum_embeddings = token_embeddings[input_mask == 1]
    embed = tf.reduce_sum(sum_embeddings, 0)/tf.reduce_sum(tf.cast(input_mask, dtype=tf.float32))

    answers = article_index.get_nns_by_vector(embed, answers_nmb)
    return [index_map[i] for i in answers]

In [7]:
question = "Боевые самолеты"
get_response(model, tokenizer, question, article_index)

['Российский истребитель пятого поколения Су-57 получит аналог ракеты «Кинжал», сообщил ТАСС источник в авиастроительной отрасли. «В соответствии с текущей ГПВ [госпрограмма вооружения] на 2018-2027 годы, гиперзвуковыми ракетами будут вооружаться истребители Су-57. Самолеты получат ракету с характеристиками, подобными ракетам "Кинжала", но она будет с внутрифюзеляжным размещением и меньшими габаритами», — сказал собеседник агентства. Источник не уточнил характеристики, название и сроки поступления на испытания новой ракеты. Первый полет Су-57 совершил в 2010 году. Минобороны России отказалось от серийных закупок машины. В ближайшие годы ведомство получит всего 12 таких истребителей. Высокоточный авиационный ракетный комплекс «Кинжал» представлен президентом России Владимиром Путиным в марте. Тогда глава государства отметил, что дальность комплекса (в случае носителя МиГ-31К) превышает 2000 километров. По имеющимся данным, ракета «Кинжал» представляет собой модернизированную версию кваз

In [9]:
question = "Красивые кошки"
get_response(model, tokenizer, question, article_index)

["Американская топ-модель и одна из «ангелов» бренда нижнего белья Victoria's Secret Тейлор Хилл опубликовала в своем Instagram-аккаунте селфи без макияжа. На фотографии видны все несовершенства кожи Хилл. Подписчики набросились на модель с критикой в комментариях. «Фу! Какое же уродливое у тебя лицо со всеми этими прыщами. Я думала, что у модели Victoria Secret должна быть идеальная кожа, но по ходу даже я могу идти на подиум. Сколько же фотошопа ты используешь на всех других фотографиях? Тебе должно быть стыдно!» — возмущались одни. «Какой ужасный снимок. Ей лучше фотографироваться с косметикой», — рекомендовали другие. «****** (Кошмар), ты без косметики такая стремная… Все бабы нафуфыренные такие…» — писали русскоязычные пользователи. Тем не менее публикация Тейлор Хилл уже собрала почти 450 тысяч лайков. В ноябре в Instagram-аккаунте @fakegirlsfvckya появился коллаж из двух фотографий американской топ-модели Кендалл Дженнер. На одной из них на девушке минимальное количество макияжа

In [10]:
question = "Торнадо"
get_response(model, tokenizer, question, article_index)

['В Парагвае корова в прыжке сбила мотоциклистку на землю. На видео произошедшего обратило внимание издание Daily Mail. Инцидент произошел в городе Капьята, департамент Сентраль на западе страны. На ролике стадо коров начинает переходить дорогу, а женщина на мотоцикле старается побыстрее проехать вперед, пока животные не перешли на другую сторону. Ее приближение вызывает агрессию у одной из коров. Она подбегает к мотоциклистке, подпрыгивает и сбивает ее задними копытами. Женщина падает, корчась от боли, проезжавший мимо на мотоцикле мужчина бросается к ней на помощь, а животное убегает в поле. По данным издания, женщине не потребовалась помощь врачей, и она не стала жаловаться в полицию на пастуха. Видео инцидента набрало тысячи просмотров в социальных сетях. Пользователи прозвали животное «кунг-фу коровой» и предположили, что она попыталась перепрыгнуть через мотоцикл.',
 ' Автоматическая станция NASA Mars InSight записала и отправила на Землю шум марсианского ветра. Видеоролик с ориг

In [11]:
question = "Известные люди"
get_response(model, tokenizer, question, article_index)

['Журнал Time огласил шорт-лист номинантов на звание «Человек года». В десятку наиболее важных персон вошли президенты России, США и Южной Кореи, а также погибший саудовский журналист Джамаль Хашкуджи. Список опубликован на сайте издания. Четверо номинантов — политические деятели разных стран. Среди них — «проходящий через кризисы» президент США Дональд Трамп и его южнокорейский коллега Мун Джэин (среди его заслуг журналисты называют проведение зимних Олимпийских игр и посредничество в переговорах США с КНДР). Также в список вошел директор ФБР Роберт Мюллер. Президент России Владимир Путин, считает Time, одержал уверенную победу на выборах, продлив свои полномочия. По их мнению, российский лидер уверенно выдерживает различные международные скандалы, в том числе ситуацию с отравлением бывшего полковника ГРУ Сергея Скрипаля и «вмешательства» в американские выборы. К наиболее влиятельным людям года Time также причислил ученого Кристин Бласи Форд, ставшую символом переживших сексуальное на